In [1]:
# All the packages needed for this project

import tkinter as tk 
from pandas import DataFrame
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import seaborn as sb
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn import neighbors
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from operator import itemgetter
from urllib.request import urlopen
from bs4 import BeautifulSoup
from sklearn.decomposition import PCA

In [2]:
# Read dataset

playerstats = pd.read_csv('Fan_Duel.csv')
playerstats

,ID,Player_name,Date,Team,Opp,H/A,GTime(ET),Team pts,Opp pts,Win-Lose,...,rb,as,st,bl,to,trey,FGM,FGA,FTM,FTA
0,5679,Drew_Eubanks,20200110,sas,mem,0,20.0,121.0,134.0,0,...,0,0,0,0,0,0,0,0,2,2
1,5679,Drew_Eubanks,20200108,sas,bos,0,19.0,129.0,114.0,1,...,0,0,0,0,0,0,1,1,0,0
2,5679,Drew_Eubanks,20191228,sas,det,1,20.5,136.0,109.0,1,...,3,0,0,0,0,0,3,3,0,0
3,5679,Drew_Eubanks,20191203,sas,hou,1,20.5,135.0,133.0,1,...,2,1,0,0,1,0,1,1,0,0
4,5679,Drew_Eubanks,20191201,sas,det,0,17.0,98.0,132.0,0,...,8,1,0,1,2,0,4,6,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15634,4817,Willie_Cauley-Stein,20191106,gsw,hou,0,19.5,112.0,129.0,0,...,4,2,0,0,0,0,1,6,2,2
15635,4817,Willie_Cauley-Stein,20191104,gsw,por,1,22.5,127.0,118.0,1,...,3,2,2,0,1,0,4,5,2,2
15636,4817,Willie_Cauley-Stein,20191102,gsw,cha,1,20.5,87.0,93.0,0,...,8,2,1,1,0,0,4,7,0,0
15637,4817,Willie_Cauley-Stein,20191101,gsw,sas,1,22.5,110.0,127.0,0,...,5,0,1,0,0,0,3,7,0,0


In [3]:
# Read dataset

data= pd.read_csv('data.csv')

In [4]:
# Assign the data to X and y

X = playerstats[['Date', 'H/A', 'GTime(ET)',
       'Team pts', 'Opp pts', 'Win-Lose','Mins_played', 'GP',
       'FDP', 'DoubleD', 'TripleD', 'FD Sal', 'FD pos', 'ADI', 'VMI',
       'pt', 'rb', 'as', 'st', 'bl', 'to', 'trey', 'FGM', 'FGA', 'FTM', 'FTA']]
y = playerstats["FDP"].values.reshape(-1, 1)
print("Shape: ", X.shape, y.shape)

Shape:  (15639, 26) (15639, 1)


In [5]:
# Use train_test_split to create training and testing data

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [6]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data

X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)


In [7]:
# Transform the training and testing data using the X_scaler and y_scaler models

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)


In [8]:
# Create the model using LinearRegression
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_scaled, y_train_scaled)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [9]:
# ElasticNet model

from sklearn.linear_model import ElasticNet


elasticnet = ElasticNet(alpha=0.6).fit(X_train_scaled, y_train_scaled)

predictions = elasticnet.predict(X_test_scaled)

MSE = mean_squared_error(y_test_scaled, predictions)
r2 = elasticnet.score(X_test_scaled, y_test_scaled)


print(f"MSE: {MSE}, R2: {r2}")
print('Model Score: %.2f' % model.score(X_train, y_train))
# print('MSE: %.2f' % mean_squared_error(y_test, predictions))
print('MAE: %.2f' % mean_absolute_error(y_test, predictions))

MSE: 0.1913732336816131, R2: 0.8137545959057331
Model Score: 0.52
MAE: 21.39


In [10]:
# Make predictions
predictions = model.predict(X_test_scaled)

model.fit(X_train_scaled, y_train_scaled)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [11]:
# Create a variable for the current season with same selected features used to train the data
current =data[[ 'Date', 'H/A', 'GTime(ET)',
       'Team pts', 'Opp pts', 'Win-Lose', 'Mins_played', 'GP',
       'FDP', 'DoubleD', 'TripleD', 'FD Sal', 'FD pos', 'ADI', 'VMI',
       'pt', 'rb', 'as', 'st', 'bl', 'to', 'trey', 'FGM', 'FGA', 'FTM', 'FTA']]

# Predict the win shares of the current dataset
predict=model.predict(current)

In [12]:
# Select the first column of data frame (player names)
player_names = playerstats.iloc[:, 1]


In [13]:
# Get the list of tuples from two lists (predicted FDP and current player names) and merge them by using zip().
combined = []
for i,j  in zip(predict, player_names):
    combined.append({'FDP_pd':i,'Player_name':j})

# Print combined 
combined

[{'FDP_pd': array([1.49244348]), 'Player_name': 'Drew_Eubanks'},
 {'FDP_pd': array([1.49244348]), 'Player_name': 'Drew_Eubanks'},
 {'FDP_pd': array([6.36486053]), 'Player_name': 'Drew_Eubanks'},
 {'FDP_pd': array([3.03899174]), 'Player_name': 'Drew_Eubanks'},
 {'FDP_pd': array([12.83123725]), 'Player_name': 'Drew_Eubanks'},
 {'FDP_pd': array([2.12919463e-09]), 'Player_name': 'Drew_Eubanks'},
 {'FDP_pd': array([1.45339741]), 'Player_name': 'Drew_Eubanks'},
 {'FDP_pd': array([0.72232755]), 'Player_name': 'Kostas_Antetokounmpo'},
 {'FDP_pd': array([0.6291767]), 'Player_name': 'Kostas_Antetokounmpo'},
 {'FDP_pd': array([2.12918346e-09]), 'Player_name': 'Kostas_Antetokounmpo'},
 {'FDP_pd': array([2.09772598]), 'Player_name': 'Frank_Mason_III'},
 {'FDP_pd': array([2.88149261]), 'Player_name': 'Frank_Mason_III'},
 {'FDP_pd': array([4.42954206]), 'Player_name': 'Frank_Mason_III'},
 {'FDP_pd': array([2.12919423e-09]), 'Player_name': 'Frank_Mason_III'},
 {'FDP_pd': array([19.90996979]), 'Player_

In [14]:
# Create a new dataframe with the predictions
new_df = pd.DataFrame(combined)
new_df

# Convert predicted values to float and round it to 2 integers
new_df['FDP_pd'] = round(new_df['FDP_pd'].astype(float),2)

# Merge the actual FDP from the current dataset to the predicted on the dataframe based on the player
new_df = new_df.merge(playerstats, on = 'Player_name', how = 'inner')

# Create a new column
# Get the difference between the predicted and actual values 
new_df['Difference'] = new_df['FDP_pd'] - new_df['FDP'] 

# Drop the current index of the DataFrame and replace it with an index of increasing integers
new_df.reset_index(drop=True)
new_df

,FDP_pd,Player_name,ID,Date,Team,Opp,H/A,GTime(ET),Team pts,Opp pts,...,as,st,bl,to,trey,FGM,FGA,FTM,FTA,Difference
0,1.49,Drew_Eubanks,5679,20200110,sas,mem,0,20.0,121.0,134.0,...,0,0,0,0,0,0,0,2,2,-0.51
1,1.49,Drew_Eubanks,5679,20200108,sas,bos,0,19.0,129.0,114.0,...,0,0,0,0,0,1,1,0,0,-0.51
2,1.49,Drew_Eubanks,5679,20191228,sas,det,1,20.5,136.0,109.0,...,0,0,0,0,0,3,3,0,0,-8.11
3,1.49,Drew_Eubanks,5679,20191203,sas,hou,1,20.5,135.0,133.0,...,1,0,0,1,0,1,1,0,0,-3.41
4,1.49,Drew_Eubanks,5679,20191201,sas,det,0,17.0,98.0,132.0,...,1,0,1,2,0,4,6,1,1,-19.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614796,14.05,Willie_Cauley-Stein,4817,20191106,gsw,hou,0,19.5,112.0,129.0,...,2,0,0,0,0,1,6,2,2,2.25
614797,14.05,Willie_Cauley-Stein,4817,20191104,gsw,por,1,22.5,127.0,118.0,...,2,2,0,1,0,4,5,2,2,-7.55
614798,14.05,Willie_Cauley-Stein,4817,20191102,gsw,cha,1,20.5,87.0,93.0,...,2,1,1,0,0,4,7,0,0,-12.55
614799,14.05,Willie_Cauley-Stein,4817,20191101,gsw,sas,1,22.5,110.0,127.0,...,0,1,0,0,0,3,7,0,0,-0.95


In [15]:
bins = [-10, 10, 20, 30, 40, 100]
group_names = ['benchPlayer','belowAvg','average','advanced','top']
new_df['ranking']= pd.cut(new_df['FDP'],bins,labels=group_names)
    

In [17]:
# Keep the first record in the dataset
FDP_predict= new_df.drop_duplicates(['Player_name'], keep = 'first')

In [25]:
FDP_predict.to_csv('fdp.csv',index=False)